# Model Evaluation Notebook
## Summary
In this notebook, I imported all of the data wrangling functions produced in the previous Analysis notebook and used them to more efficiently load and clean datasets for 2016 and 2017 data. I then trained the LGBM and Random Forest models on 2016 data, before validating individual and ensemble results on unseen 2017 data from Q1 and Q2—the most recent data available. I selected the best performing model on the validation set, and then used it to create a final test-set accuracy metric.

## Results
The accuray metrics for the LGBM, Random Forest, and LGBM+Random Forest on the 2017 validation set were:
- LGBM: 0.659
- Random Forest: 0.648
- LGBM+RF Ensemble: 0.659

Based on the above results, I chose the LGBM model due to its higher accuracy compared to Random Forest and its speed over the LGBM+RF Ensemble.

The final test prediction accuracy for the LGBM on 2017 test set was 65.9%.

In [1]:
# saved functions from other notebook
from data_wrangle_functions import *

# general use
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from my_plot import PrettyPlot
PrettyPlot(plt);

In [2]:
%store -r stations

# 1. Data Wrangling Functions
The functions below combine all of the data cleaning and feature engineering steps that I discussed in the Analysis notebook. The workflow consists of:
1. Loading trips data and merging with stations data already containing the cluster information.
2. Adding the target data using Great Circle and Google Maps.
3. Removing unnecessary columns, imputing missing values, and formatting datetime series.
4. Adding information about hourly temperature.
5. Converting the date and time information to cyclical.

These general functions will be used to import and format the 2016 training data and 2017 validation / testing data.

In [3]:
def data_loading(stations, path):
    
    trips = data_load(path)
    trips, _ = trips_station_merge(trips, stations)
    
    return trips

In [4]:
def add_target(trips, max_dist, min_dist, n_samples, targets_inst, fit=True):
    
    trips = great_circle_calc(trips)
    
    if fit == True:
        distances_df = gmaps_distance(trips, max_dist=max_dist, min_dist=min_dist, samples=n_samples)
        targets_inst, _ = target_creation(distances_df).fit()
        targets_inst.transform(trips);
    else:
        pass
        
    return trips, targets_inst

In [5]:
def clean_and_format(trips, col_to_drop, date_format, is_2016=False):
    
    trips = feature_cleaning(trips, col_to_drop)
    
    if is_2016 == True:
        trips = fix_2016(trips)
    else:
        pass
    
    format_date_inst = format_dates(trips, date_formats=date_format)
    format_date_inst.add_holidays(trips)
    format_date_inst.create_day_week_hour(trips, date_formats=['%Y-%M-%D %H:%M:%S', '%m/%d/%Y'])
    format_date_inst.specify_weekend(trips);
    
    trips = column_rename(trips)
    
    return trips

In [6]:
def add_weather(trips, start_date, end_date):
    
    weather_data = obtain_weather(date_start=start_date, date_end=end_date)
    trips = add_weather_feature(trips, weather_data)
    
    return trips

In [7]:
def time_cycles(trips):
    
    ct = convert_time(trips)
    
    ct.cyclical_hour(trips)
    ct.cyclical_month(trips)
    ct.cyclical_day(trips)
    ct.cyclical_day_of_week(trips)
    
    return trips

# 2. Load and Clean Data

## 2.1 2016 Train Data

In [8]:
trips_2016 = data_loading(stations, 'trips2016/')

In [9]:
trips_2016, targets_inst = add_target(trips_2016, max_dist=2.5, min_dist=1.0,
                                     n_samples=2000, targets_inst=None, fit=True)

In [10]:
col_drop = ['bikeid', 'stoptime', 'tripduration', 'from_station_name', 
            'to_station_id', 'to_station_name', 'id_end', 'id_start', 'cluster_end', 
            'latitude_start', 'longitude_start', 'latitude_end', 'longitude_end', 
            'dpcapacity_end', 'online_date_end', 'circle_dist', 'gmaps_dist']


date_formats = ['%Y-%M-%D %H:%M:%S', '%m/%d/%Y']

In [11]:
trips_2016 = clean_and_format(trips_2016, col_drop, date_format=date_formats, is_2016=True)

In [12]:
trips_2016 = add_weather(trips_2016, start_date='201601', end_date='201701')

In [13]:
trips_2016 = time_cycles(trips_2016)

In [14]:
dummy_cols = ['usertype', 'gender', 'cluster_start']

trips_2016 = trips_2016.dropna()
X_2016 = one_hot_encode(trips_2016, dummy_cols)

In [15]:
y_2016 = trips_2016.shorter_two
df_info(X_2016)

#samples: 3594737 , #features: 121


,trip_id,birthyear,dpcapacity_start,holiday,station_age,is_weekend,temperature,start_hour_sin,start_hour_cos,month_sin,...,cluster_90,cluster_91,cluster_92,cluster_93,cluster_94,cluster_95,cluster_96,cluster_97,cluster_98,cluster_99
0,8547211,1965.0,15,True,260,0,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,8547214,1981.0,15,True,878,0,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,8547215,1994.0,15,True,828,0,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,8547216,1982.0,15,True,238,0,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,8547217,1976.0,15,True,238,0,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## 2.2 2017 Validate and Test Data

In [16]:
trips_2017 = data_loading(stations, 'trips2017/')
trips_2017 = trips_2017.rename(columns={'end_time': 'stoptime'})
trips_2017 = trips_2017.dropna(subset=['cluster_start', 'cluster_end'])

In [17]:
trips_2017, _ = add_target(trips_2017, max_dist=2.5, min_dist=1.0,
                       n_samples=2000, targets_inst=targets_inst, fit=False)

targets_inst.transform(trips_2017);

In [18]:
date_formats = ['%m/%d/%Y %H:%M:%S', '%m/%d/%Y']

trips_2017 = clean_and_format(trips_2017, col_drop, date_format=date_formats, is_2016=False)

In [19]:
trips_2017['start_time'] = pd.to_datetime(trips_2017.start_time, format='%m/%d/%Y %H:%M:%S')
trips_2017 = add_weather(trips_2017, start_date='201701', end_date='201707')

In [20]:
trips_2017 = time_cycles(trips_2017)

In [21]:
trips_2017 = trips_2017.dropna()
X_2017 = one_hot_encode(trips_2017, dummy_cols)

In [22]:
y_2017 = trips_2017.shorter_two
df_info(X_2017)

#samples: 1547163 , #features: 121


,trip_id,birthyear,dpcapacity_start,holiday,station_age,is_weekend,temperature,start_hour_sin,start_hour_cos,month_sin,...,cluster_90.0,cluster_91.0,cluster_92.0,cluster_93.0,cluster_94.0,cluster_95.0,cluster_96.0,cluster_97.0,cluster_98.0,cluster_99.0
0,12979230,1984.0,15.0,True,403,1,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,12979231,1984.0,15.0,True,1289,1,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,12979232,1985.0,15.0,True,114,1,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,12979233,1990.0,27.0,True,1206,1,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,12979234,1990.0,19.0,True,1200,1,25.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# 3. LightGBM and Random Forest Training

Below, I trained the LGBM and Random Forest models on the entire 2016 dataset. I used the best parameters found for the LGBM model (n_estimators=2000, learning_rate=0.125), but slightly reduced the number of estimators in the Random Forest model to improve generalization.

## 3.1 LightGBM

In [23]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=2000, learning_rate=0.125)
lgbm.fit(X_2016, y_2016)

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0,
        learning_rate=0.125, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
        random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=50000, subsample_freq=1)

## 3.2 Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300)
rf.fit(X_2016, y_2016)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# 4. Validation and Testing on 2017 data
In this section, I evaluated my models on the most current 2017 data to ensure that they will provide reasonable accuracy on unseen data for future-looking predictions. I created a validation set to choose between using the LGBM only, Random Forest only, or an ensemble average between the two for the final model selection. The best performing model is then evaluated on the final 2017 test set.

## 4.1 Creating equal-sized validation and test sets for 2017 data

In [27]:
from sklearn.model_selection import train_test_split

X_validate, X_test, y_validate, y_test = train_test_split(X_2017, y_2017, 
                                                          test_size=0.5, random_state=1)

## 4.2 Evaluating LGBM only, RF only, and Ensemble on validation set

In [30]:
from sklearn.metrics import accuracy_score

predict_lgbm = lgbm.predict(X_validate)
print('LGBM Validation Accuracy:',
      round(accuracy_score(predict_lgbm, y_validate), 3))

LGBM Validation Accuracy: 0.659


In [31]:
from sklearn.metrics import accuracy_score

predict_rf = rf.predict(X_validate)
print('Random Forest Validation Accuracy:',
      round(accuracy_score(predict_rf, y_validate), 3))

Random Forest Validation Accuracy: 0.648


The ensemble average function below calculates the prediction probability and then averages the probabilities before being converted to the binary prediction. I chose this method because I only had two models with high enough accuracy to consider for the ensemble. I tried to add the logistic regression model and use a majority-vote ensemble method, but the lower accuracy of the logistic regression model seemed to reduce accuracy.

In [32]:
def ensemble_average(X_to_predict, models):
    
    results = []
    for model in models:

        proba = model.predict_proba(X_to_predict)
        results.append(proba[:,0])
    
    avg_proba = np.mean([results[1], results[0]], axis=0)
    predictions = np.where(avg_proba > 0.5, 0, 1)
    
    return avg_proba, predictions

In [33]:
models = [lgbm, rf]
avg_proba, predictions = ensemble_average(X_validate, models)

In [34]:
print('Ensemble Validation Accuracy:',
      round(accuracy_score(predictions, y_validate), 3))

Ensemble Validation Accuracy: 0.659


## 4.3 Final Test Set Accuracy
Because the LGBM model gives the highest accuracy out of the 3 evaluated validation models, I decided to move forward with it alone. It also has the benefit of being signficantly faster than other tested models. The final accuracy score on the 2017 test data is shown below as 65.9%.

In [36]:
predict_lgbm = lgbm.predict(X_test)
print('LGBM Test Accuracy:',
      round(accuracy_score(predict_lgbm, y_test), 3))

LGBM Test Accuracy: 0.659


# 5. Final Thoughts
Interestingly, the Random Forest model went from having the highest prediction accuracy on the cross-validated 2016 dataset in the previous notebook (~68% compared to 67%), to having slightly less accuracy than the LGBM model on unseen 2017 data. Given more time / computing resources, it would be beneficial to explore learning curves for the Random Forest model and use those results to guide hyperparameter optimization. 

Further, I think that trying to obtain an additional (or three) models that perform on-par with these two would be useful for creating a majority vote ensemble, or using a stacker to better improve individual model performance. That being said, the LGBM provides reasonable accuracy for this problem, while also excelling in terms of speed and computing cost.